# Generate male PCs

In [ ]:
plink --bfile rbc --keep sample_males_osmotic.txt --make-bed --out rbc.males
plink --bfile rbc --keep sample_females_osmotic.txt --make-bed --out rbc.females


In [ ]:
R

require(gdsfmt)
require(SNPRelate)

##### Following http://corearray.sourceforge.net/tutorials/SNPRelate/#format-conversion-from-plink-binary-files #####

  # Format conversion from PLINK binary files #####
  bed.fn <- "rbc.males.bed"
  fam.fn <- "rbc.males.fam"
  bim.fn <- "rbc.males.bim"
  snpgdsBED2GDS(bed.fn, fam.fn, bim.fn, "rbc.males.gds")
  
  # Data Analysis #####
  genofile <- snpgdsOpen("rbc.males.gds")
  
  # LD-based SNP pruning #####
  set.seed(1000)
  snpset <- snpgdsLDpruning(genofile, ld.threshold=0.2)
  
  snpset.id <- unlist(snpset)
  
  # Principal Component Analysis (PCA) #####
  pca <- snpgdsPCA(genofile, snp.id=snpset.id, num.thread=8)
  
  # save.image("WorkingProgress.RData")
  
  # variance proportion (%)
  pc.percent <- pca$varprop*100
  head(round(pc.percent, 2), n=10)
  
  # make a data.frame
  tab <- data.frame(sample.id = pca$sample.id,
                    EV1 = pca$eigenvect[,1],    # the first eigenvector
                    EV2 = pca$eigenvect[,2],    # the second eigenvector
                    EV3 = pca$eigenvect[,3],    # the first eigenvector
                    EV4 = pca$eigenvect[,4],    # the second eigenvector
                    EV5 = pca$eigenvect[,5],    # the first eigenvector
                    EV6 = pca$eigenvect[,6],    # the second eigenvector
                    EV7 = pca$eigenvect[,7],    # the first eigenvector
                    EV8 = pca$eigenvect[,8],    # the second eigenvector
                    EV9 = pca$eigenvect[,9],    # the first eigenvector
                    EV10 = pca$eigenvect[,10],    # the second eigenvector
                    stringsAsFactors = FALSE)
  head(tab)
  
  # Draw
#   plot(tab$EV2, tab$EV1, xlab="eigenvector 2", ylab="eigenvector 1")
#   plot(tab$EV4, tab$EV3, xlab="eigenvector 2", ylab="eigenvector 1")
#   plot(tab$EV6, tab$EV5, xlab="eigenvector 2", ylab="eigenvector 1")
  
write.table(tab, "rbc.males.ex_related.EVs.txt", row.names = FALSE, quote = FALSE)
  # tab <- read.table(paste0(race, ".EVs.txt"), stringsAsFactors = FALSE, header = TRUE)



# GWAS for osmotic hemolysis in males

In [ ]:
#!/bin/sh

working_dir=/share/storage/REDS-III/rbc_male_osmotic
imputation_root=/share/storage/REDS-III/RBCOmics/data/imputation/v1/imputations_all_maf0.01/
phenotype_root=/share/storage/REDS-III/GWA_G_by_Sex/rbc_male_osmotic/phenotype

method=palinear

for chr in {1..23};do
    out_file=rbc.males.1000G_p3.chr$chr.osmotic~age+race+donfreq+evs+SNP.stats
    phenotype_file=pheno_males_osmotic.txt
    geno_prefix=rbc.ALL.1000G_p3.chr

    /share/storage/REDS-III/common/software/nextflow/nextflow-0.25.1-all \
        /share/storage/REDS-III/common/software/pipelines/_pipeline.association.out_stats_files.v0.1.nf \
            --final_chunks $imputation_root/chunks/final_chunks.chr$chr \
            --input_pheno $phenotype_root/$phenotype_file \
            --imputation_dir $imputation_root/chr$chr \
            --example_mldose $imputation_root/chr$chr/rbc.ALL.1000G_p3.chr$chr.1.mach.mldose.gz \
            --geno_prefix $geno_prefix \
            --working_dirs $working_dir \
            --out $working_dir/$out_file \
            --method $method 
done  



## Generate plots

In [ ]:
basedir=/share/storage/REDS-III/GWA_G_by_Sex/rbc_male_osmotic
prefix=rbc.males.1000G_p3
suffix=osmotic~age+race+donfreq+evs+SNP
cd $basedir

# Filter out variants with MAF <= 0.01 in study
for inFile in ${prefix}*.stats; do
    outFile=$basedir/final/${inFile%".stats"}.maf_gt_0.01
    echo Processing $inFile
    
    echo "name position A1 A2 Freq1 MAF Quality Rsq n Mean_predictor_allele beta_SNP_add sebeta_SNP_add chi2_SNP chi p or_95_percent_ci" \
        > $outFile

    tail -n +1 $inFile | \
        perl -slane 'if ($F[5] >= 0.01 && $F[7] >= 0.8) { print "$_"; }' \
        >> $outFile
done


In [ ]:
### START Generate plots ###

outFile=$basedir/processing/${prefix}.${suffix}.maf_gt_0.01.table
echo -e "VARIANT_ID\tCHR\tPOSITION\tP\tTYPE" > $outFile
for chr in {1..23}; do
    inFile=$basedir/final/${prefix}.chr${chr}.${suffix}.maf_gt_0.01
    echo Processing $inFile
    awk -v CHR=$chr 'NR>1{if (($5 eq "A" || $5 eq "C" || $5 eq "G" || $5 eq "T") && ($4 eq "A" || $4 eq "C" || $4 eq "G" || $4 eq "T")) {
                        print $1"\t"CHR"\t"$2"\t"$15"\tsnp";
                    } else {
                        print $1"\t"CHR"\t"$2"\t"$15"\tindel";
                    }}' $inFile  >> $outFile
done




In [ ]:
/share/storage/REDS-III/scripts/qsub_job.sh \
    --job_name gwas_plots \
    --script_prefix $basedir/processing/${prefix}.${suffix}.plot \
    --mem 10 \
    --cpu 1 \
    --priority 0 \
    --program /share/storage/REDS-III/common/software/R/generate_gwas_plots.v9.R \
    --in $basedir/processing/${prefix}.${suffix}.maf_gt_0.01.table \
    --in_chromosomes autosomal_nonPAR \
    --in_header \
    --out $basedir/final/${prefix}.${suffix}.maf_gt_0.01 \
    --col_id VARIANT_ID \
    --col_chromosome CHR \
    --col_position POSITION \
    --col_p P \
    --col_variant_type TYPE \
    --generate_snp_indel_manhattan_plot \
    --manhattan_odd_chr_color red \
    --manhattan_even_chr_color blue \
    --manhattan_points_cex 1.5 \
    --manhattan_cex_axis 2 \
    --manhattan_cex_lab 2 \
    --generate_snp_indel_qq_plot \
    --qq_lines \
    --qq_points_bg black \
    --qq_lambda

### END Generate plots ###



# Heritability calculation using GCTA

In [ ]:
awk '{print $1"\t"$1"\t"$2}' \
    /share/storage/REDS-III/GWA_G_by_Sex/rbc_male_osmotic/phenotype/pheno_males_osmotic.txt \
    > pheno_males_osmotic.txt

In [ ]:
# generate grm
/share/storage/REDS-III/Heritability/gcta_1.92.2beta/gcta64 \
    --bfile /share/storage/REDS-III/GWA_G_by_Sex/rbc_male_osmotic/PCA/rbc.males \
    --make-grm --out rbc.males --thread-num 16


In [ ]:
# calculate heritability
/share/storage/REDS-III/Heritability/gcta_1.92.2beta/gcta64 \
    --reml --grm rbc.males --pheno pheno_males_osmotic.txt \
    --out rbc.males.osmotic.heritability --threads 16

# Prepare summary stats for LDSC

In [ ]:
cd /share/storage/REDS-III/GWA_G_by_Sex/rbc_male_osmotic/heritability

echo -e "MarkerName\tCHR\tPOS\tAllele1\tAllele2\tEffect\tStdErr\tP-value\t" > rbc.male.osmotic~age+race+donfreq+evs+SNP.maf_gt_0.01.summaryStats.txt

for chr in {1..23};do 
    awk -v CHR=$chr 'NR>1{print $1"\t"CHR"\t"$2"\t"$3"\t"$4"\t"$11"\t"$12"\t"$15}' \
       /share/storage/REDS-III/GWA_G_by_Sex/rbc_male_osmotic/final/rbc.males.1000G_p3.chr${chr}.osmotic~age+race+donfreq+evs+SNP.maf_gt_0.01 \
      >> rbc.male.osmotic~age+race+donfreq+evs+SNP.maf_gt_0.01.summaryStats.txt
done


gzip rbc.male.osmotic~age+race+donfreq+evs+SNP.maf_gt_0.01.summaryStats.txt


# Estimate heritability for osmotic hemolysis in males

In [ ]:
sudo docker run -v /data:/data \
    --rm rtibiocloud/genomic_sem:v1_75a57cd Rscript /opt/GenomicSEM_commonFactor.R \
    --input_files /data/rbc.male.osmotic~age+race+donfreq+evs+SNP.maf_gt_0.01.summaryStats.txt.gz \
    --trait_names male_osmotic \
    --sample_sizes 6128 \
    --sample_prev NA \
    --pop_prev NA \
    --reference /data/reference.1000G.maf.0.005.txt \
    --info_filter 0.8 \
    --maf_filter 0.01 \
    --out_prefix /data/ \
    --ld /data/eur_w_ld_chr \
    --munge TRUE \
    --LDSC TRUE
    
